In [2]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install tqdm
!pip install tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Load Data Set

In [6]:
from datasets import load_dataset

# Assuming you have a plain text file called 'custom_data.txt'
dataset = load_dataset('text', data_files='custom_data.txt')


Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `ashudalviTK` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authentic

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# Load the GPT-2 tokenizer and model
model_name = "gpt2"  # Changed to "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True) # Added use_auth_token=True for authentication
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True) # Added use_auth_token=True for authentication

# Add padding token to the tokenizer
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Update model embeddings



/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [18]:
def tokenize_function(examples):
    # Add a padding token if it doesn't exist
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # Now you can safely pad the sequences
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/22 [00:00<?, ? examples/s]

In [19]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,  # You can increase this if you have more data or more computational power
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    evaluation_strategy="epoch",
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
# Tokenize with consistent padding and truncation
def tokenize_function(examples):
    # Tokenize the text and create labels (targets) for language modeling
    tokenized_output = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    # Create labels by shifting the input_ids one position to the right
    tokenized_output["labels"] = tokenized_output["input_ids"].copy()
    return tokenized_output

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4)

# Define a function to remove empty examples
def remove_empty_examples(example):
    """Check if the 'text' field is empty after tokenization"""
    return len(example['input_ids']) > 0 # Updated field name to input_ids

# Remove empty examples using the newly defined function
cleaned_dataset = tokenized_dataset.filter(remove_empty_examples)

# Split dataset
if "test" not in cleaned_dataset:
    cleaned_dataset = cleaned_dataset["train"].train_test_split(test_size=0.1)

train_dataset = cleaned_dataset["train"]
eval_dataset = cleaned_dataset["test"]

# Use a data collator for padding
from transformers import DataCollatorForLanguageModeling # Changed to DataCollatorForLanguageModeling
# Explicitly set mlm=False to disable Masked Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # Changed to DataCollatorForLanguageModeling and set mlm=False

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator  # Use data collator for dynamic padding
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,4.613007
2,No log,4.608077
3,No log,4.599567


TrainOutput(global_step=15, training_loss=4.144854990641276, metrics={'train_runtime': 588.8769, 'train_samples_per_second': 0.097, 'train_steps_per_second': 0.025, 'total_flos': 14893645824000.0, 'train_loss': 4.144854990641276, 'epoch': 3.0})

In [24]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [26]:
from transformers import pipeline

# Load the fine-tuned model
generator = pipeline("text-generation", model="./fine_tuned_model", tokenizer=tokenizer)

# Generate text based on a prompt
prompt = "what is copmuting"
output = generator(prompt, max_length=100, num_return_sequences=1)
print(output[0]['generated_text'])


what is copmuting it all over the place in his head) by the way, how dare you put all these things and even some of his own stuff in the game.


It's funny, so not really in the slightest funny.


So here I went... The point is that the point is to try and make the game as beautiful as possible. I haven't changed anything and I have not changed anything but now the game itself is just too much and there's no way
